After fixing my dumb mistake in the bispy package, the code here will work now. This notebook uses a similar process to what we worked out to build the name list to work with from the ITIS data. I've provided a few notes on what's going on for your reference.

In [1]:
import json
import bispy
from IPython.display import display
from collections import Counter

sgcn = bispy.sgcn.Search()

In [2]:
with open('cache/itis.json', 'r') as f:
    itis_cache = json.loads(f.read())
    f.close()

I also fixed the problem where those two scientific names were not getting routed to the appropriate ITIS search. I re-ran the ITIS cache process and dumped out a new file. The following show the number of records in the raw cache and then the number of records if we select for only successful records. They show the same number now.

In [3]:
len(itis_cache)

116

In [4]:
len([i for i in itis_cache if i["processing_metadata"]["status"] == "success"])

116

Because we know every record is usable, we can make the process of building our list simpler. We don't need the conditional piece.

In [5]:
lookup_name_list = list()
for itis_doc_set in itis_cache:
    for itis_doc in itis_doc_set["itis_data"]:
        lookup_name_list.append({
            "scientific_name": itis_doc["nameWInd"],
            "itis_usage": itis_doc["usage"]
        })


We see here now that we have more names to look up because we are using both valid and invalid/accepted and not accepted names from ITIS.

In [6]:
len(lookup_name_list)

122

We can now run the loop over the names the way we were going to and produce a set of information from one of the sources like the SGCN case we started with. In this case, I pass in the "itis_usage" property as an optional variable called name_source to this function where it comes back with the data. This lets us look to see what all cases were found with the invalid vs. valid ITIS names below. I go ahead and put the data from SGCN into their own list of documents so that we can look at the results again. This will take a few seconds to run because we are working through each record sequentially.

In [7]:
%%time
sgcn_info = []
for name in lookup_name_list:
    sgcn_info.append(sgcn.search(name["scientific_name"], name_source=name["itis_usage"]))

CPU times: user 2.47 s, sys: 205 ms, total: 2.67 s
Wall time: 1min 5s


In [8]:
sgcn_info

[{'processing_metadata': {'status': 'failure',
   'date_processed': '2019-08-02T20:17:12.923217',
   'status_message': 'Not Matched',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Procapra gutturosa'},
  'parameters': {'Scientific Name': 'Procapra gutturosa',
   'Name Source': 'valid'}},
 {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-02T20:17:13.566322',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Salvelinus namaycush'},
  'parameters': {'Scientific Name': 'Salvelinus namaycush',
   'Name Source': 'valid'},
  'sgcn_species': {'statelist_2005': 'Illinois,Maine,Montana,New Hampshire,Ohio,Vermont',
   'statelist_2015': 'Alaska,Illinois,Maine,Montana,New Hampshire,New York,Vermont',
   'scientificname': 'Salvelinus namaycush',
   'commonname': 'lake trout',
   'taxonomicgroup': 'Fish',
   'taxonomicrank': 'Species',
   'matchmethod': 'Exact Ma

This is another utility function that can be used to quickly count how many records in a list like this fit into different categories. The following checks for how many total species we found in the SGCN list vs. not by looking at the processing_metadata/success property.

In [9]:
Counter(spp["processing_metadata"]["status"] for spp in sgcn_info)

Counter({'failure': 65, 'success': 57})

This counter looks at the values that we passed in from itis_usage to see how those are distributed across the successful results. It shows us that all 57 we matched to SGCN were done using the valid or accepted names.

In [10]:
Counter(spp["parameters"]["Name Source"] for spp in [r for r in sgcn_info if r["processing_metadata"]["status"] == "success"])

Counter({'valid': 50, 'accepted': 7})

Now we can look to see which WLCI species have been studied (or at least showed up in literature) that are on the Wyoming SGCN list. This is kind of a clunky way to do this query, but it works for our purposes. This doesn't tell us a whole lot, but it does show that USGS funding has been contributed to some of the species that the state of Wyoming considers to be a priority for conservation research and management.

In [11]:
sgcn_wy_names = []
for spp in [r["sgcn_species"] for r in sgcn_info if r["processing_metadata"]["status"] == "success"]:
    if "Wyoming" in spp["statelist_2005"] or "Wyoming" in spp["statelist_2015"]:
        print(spp["commonname"], '======', spp["scientificname"])
        sgcn_wy_names.append(spp["scientificname"])

bighorn sheep ====== Ovis canadensis
cutthroat trout ====== Oncorhynchus clarkii
Uinta Ground Squirrel ====== Urocitellus armatus
Sage Thrasher ====== Oreoscoptes montanus
Bell's Sparrow ====== Artemisiospiza belli
mottled sculpin ====== Cottus bairdii
Brown Bear ====== Ursus arctos
Bald Eagle ====== Haliaeetus leucocephalus
moose ====== Alces alces
Brewer's Sparrow ====== Spizella breweri
Ferruginous Hawk ====== Buteo regalis
Pygmy Rabbit ====== Brachylagus idahoensis
mountain sucker ====== Catostomus platyrhynchus
Greater Sage Grouse ====== Centrocercus urophasianus
Colorado River cutthroat trout ====== Oncorhynchus clarkii pleuriticus


Just to take this one step further (and because this is all "our" data), we can also look to see what Wyoming SGCN species from the latest 2015 data are not in the WLCI list. Of course, we don't know that these species actually occur in the WLCI geographic area, so we'd need to do some more work on making this at all meaningful. Our API for SGCN species is really clunky right now, so don't get too deep into this one.

In [12]:
import requests

r_sgcn_api = requests.get("https://api.sciencebase.gov/bis-api/api/v1/swap/state?state=Wyoming").json()

for hit in r_sgcn_api["hits"]["hits"]:
    if hit["_source"]["properties"]["sgcn2015"] == 1 and hit["_source"]["properties"]["sgcn2015"] not in sgcn_wy_names:
        print(hit["_source"]["properties"]["commonname"], "=====", hit["_source"]["properties"]["scientificname"])

Bobolink ===== Dolichonyx oryzivorus
Trumpeter Swan ===== Cygnus buccinator
Midget Faded Rattlesnake ===== Crotalus oreganus concolor
mule-eared bat ===== Corynorhinus townsendii
moose ===== Alces alces
Yellow-billed Cuckoo ===== Coccyzus americanus
Western Painted Turtle ===== Chrysemys picta bellii
finescale dace ===== Chrosomus neogaeus
Black Tern ===== Chlidonias niger
Rubber Boa ===== Charina bottae
Mountain Plover ===== Charadrius montanus
Hispid Pocket Mouse ===== Chaetodipus hispidus
Greater Sage Grouse ===== Centrocercus urophasianus
flannelmouth sucker ===== Catostomus latipinnis
bluehead sucker ===== Catostomus discobolus
central stoneroller ===== Campostoma anomalum
devil crayfish ===== Cambarus diogenes
Chestnut-collared Longspur ===== Calcarius ornatus
Lark Bunting ===== Calamospiza melanocorys
Swainson's Hawk ===== Buteo swainsoni
Ferruginous Hawk ===== Buteo regalis
Boreal Owl ===== Aegolius funereus
Barrow's Goldeneye ===== Bucephala islandica
Pygmy Rabbit ===== Brachy